In [1]:
import torch
import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler

# Load the Wine Quality dataset
wine = load_wine()
X, y = wine.data, wine.target

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the Bayesian logistic regression model
def model(X, y):
    num_features = X.shape[1]
    weight = pyro.sample('weight', dist.Normal(torch.zeros(num_features), torch.ones(num_features)))
    intercept = pyro.sample('intercept', dist.Normal(0, 10))
    logits = torch.matmul(X, weight) + intercept
    with pyro.plate('data', len(X)):
        pyro.sample('obs', dist.Bernoulli(logits=logits), obs=y)

# Define the guide (variational distribution)
def guide(X, y):
    num_features = X.shape[1]
    slope_mean = pyro.param('slope_mean', torch.zeros(num_features))
    slope_std = pyro.param('slope_std', torch.ones(num_features), constraint=dist.constraints.positive)
    intercept_mean = pyro.param('intercept_mean', torch.tensor(0.))
    intercept_std = pyro.param('intercept_std', torch.tensor(1.), constraint=dist.constraints.positive)
    pyro.sample('weight', dist.Normal(slope_mean, slope_std))
    pyro.sample('intercept', dist.Normal(intercept_mean, intercept_std))

# Set up the optimizer and the SVI object
optimizer = Adam({"lr": 0.01})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# Run variational inference
num_steps = 1000
pyro.clear_param_store()
for step in range(num_steps):
    loss = svi.step(X, y)
    if step % 100 == 0:
        print(f"Step {step}/{num_steps}, Loss: {loss}")

# Get the posterior distribution parameters
slope_mean = pyro.param('slope_mean').detach().numpy()
slope_std = pyro.param('slope_std').detach().numpy()
intercept_mean = pyro.param('intercept_mean').item()
intercept_std = pyro.param('intercept_std').item()

print("Posterior slope mean:", slope_mean)
print("Posterior slope standard deviation:", slope_std)
print("Posterior intercept mean:", intercept_mean)
print("Posterior intercept standard deviation:", intercept_std)

ModuleNotFoundError: No module named 'pyro'